## calculate the Metabolic Index for each grid point at each depth for FOSI hindcast

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]
filename = f'{path}/DPLE_driftcorrected_TEMP_mean_ens_mean.nc'
ds1 = xr.open_dataset(filename)
Temp = ds1['TEMP'].isel(Y=0, z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2))
#
filename = f'{path}/DPLE_driftcorrected_pO2_mean_dimension_reduced_ens_mean_monthly.nc'
ds2 = xr.open_dataset(filename)
pO2 = ds2['pO2']
Temp

<xarray.DataArray 'TEMP' (L: 122, z_t: 35, nlat: 140, nlon: 195)>
[116571000 values with dtype=float64]
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 ...
    ULONG    (nlat, nlon) float64 ...
    ULAT     (nlat, nlon) float64 ...
    TLONG    (nlat, nlon) float64 ...
    Y        int64 1954
  * L        (L) int64 1 2 3 4 5 6 7 8 9 ... 114 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon

## Calculate Φ using 9 different combinations of Ac, Eo from Traits_db

In [3]:
import traits_db
from scipy import stats as scistats

In [4]:
def attrs_label(attrs): 
    """generate a label from long_name and units"""
    return f'{attrs["long_name"]} [{attrs["units"]}]'

In [5]:
df = traits_db.df_mi
df.head()

,Species,Phylum,Ao,Ac,Eo
0,Acanthephyra acutifrons,Crustacea,26.809651,13.364732,-0.0278
1,Acanthephyra curtirostris,Crustacea,25.575448,14.364194,0.0380
2,Acanthephyra purpurea,Crustacea,23.310023,7.460879,0.1320
3,Acanthephyra smithi,Crustacea,28.985507,17.657939,-0.0301
5,Acipenser brevirostrum,Chordata,22.123894,4.901716,0.3626


### Plot trait histograms

In [6]:
Ac_dist = np.percentile(df.Ac, [50])
#Ao_dist = np.percentile(df.Ao, [25, 50, 75])
Eo_dist = np.percentile(df.Eo, [25, 50, 75])
print(Ac_dist)
#print(Ao_dist)
print(Eo_dist)

[7.34832416]
[0.1985 0.3389 0.5625]


### calculate Φ based on the 9 combinations

In [7]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product
dss = xr.Dataset()

for i, j in product(range(nrow), range(ncol)):
    Ac = Ac_dist[i]
    Eo = Eo_dist[j]
    print(Ac,Eo)
    dss[f'Phi_{nrow*i+j}'] = mi.Phi(pO2, Temp, Ac, Eo, dEodT=0.022)
    # = xr_reshape(ppphi,'time',['year','mon'],[np.arange(70),np.arange(12)]).mean(dim='mon')

7.3483241595629805 0.1985
7.3483241595629805 0.3389
7.3483241595629805 0.5625
CPU times: user 4.52 s, sys: 8.14 s, total: 12.7 s
Wall time: 31 s


In [8]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dss.load()

<xarray.Dataset>
Dimensions:  (z_t: 35, nlat: 140, nlon: 195, L: 122)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 14.96 14.96 14.97 14.97 ... 70.43 70.42 70.42
    ULONG    (nlat, nlon) float64 101.7 102.9 104.0 105.1 ... 318.9 319.5 320.0
    ULAT     (nlat, nlon) float64 15.13 15.13 15.14 15.14 ... 70.64 70.63 70.63
    TLONG    (nlat, nlon) float64 101.2 102.3 103.4 104.5 ... 318.6 319.2 319.7
    Y        int64 1954
  * L        (L) int64 1 2 3 4 5 6 7 8 9 ... 114 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_0    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_1    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_2    (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan

In [9]:
%%time
dss.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_mean_dimension_reduced_ens_mean_monthly.nc', mode='w')

CPU times: user 33.5 ms, sys: 1.03 s, total: 1.06 s
Wall time: 1.32 s
